#**Inteligencia de negocios - ISIS3301**

#**Proyecto 1 - Analitica de texto**

##**Sección 2**
##**Grupo 23**
*   Rafael Santiago Bastos Russi - *202110792*
*   David Santiago Valderrama Herrera - *201910987*
*   Jesús Alejandro Dávila Pinchao - *202014263*




# **Caso de estudio**

La Organización de las Naciones Unidas (ONU) adopta, el 25 de septiembre del año 2015, la
Agenda 2030i para el desarrollo sostenible, cuyo fin es reducir la pobreza, garantizar acceso
a la salud y educación, buscar igualdad de género y oportunidades, disminuir el impacto
ambiental, entre otros. Esta agenda se basa en 17 objetivos de desarrollo sostenibleii (ODS)
y 169 metas (derivadas de los diferentes ODS).
Dentro del trabajo en conjunto de diferentes entes para alcanzar el cumplimiento de los
ODS, muchas entidades tienen como enfoque el seguimiento y la evaluación de las políticas
públicas y su impacto a nivel social. Este es el caso del Fondo de Poblaciones de las Naciones
Unidas (UNFPAiii) que, junto con entidades públicas y haciendo uso de diferentes
herramientas de participación ciudadana, busca identificar problemas y evaluar soluciones
actuales, relacionando la información con los diferentes ODS. En este contexto, uno de los
procesos que requiere de un mayor esfuerzo es la clasificación de la información textual
que es recopilada, ya que es una tarea que consume gran cantidad recursos y para la cual
se requiere un experto. Es por esto que, en los últimos años, UNFPA ha venido trabajando,
en conjunto con la Universidad de los Andes, en la implementación de diferentes estrategias
de clasificación de textos, que les permitan hacer un análisis automatizado de opiniones
que representan la voz de los habitantes locales sobre problemáticas de su entorno
particular.
Para apoyar a UNFPA en este proceso se ha planteado el primer proyecto del curso, cuyo
objetivo es desarrollar un modelo de clasificación, con técnicas de aprendizaje
automático, que permita relacionar de manera automática un texto según los ODS. Al
igual que desarrollar una aplicación que facilite la interacción con el resultado de dicho
modelo. El modelo podrá ser utilizado entonces para la interpretación y análisis de la
información textual que es recopilada a través de diferentes fuentes por UNFPA en procesos
de planeación participativa para el desarrollo a nivel territorial.

#**Etapa 1**

## **1. Importación de librerias**

In [3]:
!pip install num2words

Defaulting to user installation because normal site-packages is not writeable


In [42]:
import pandas as pd
import numpy as np
import sys
import re, string, unicodedata

import inflect
import nltk
import spacy
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score

# Versiones anteriores a 1.2 de sklearn: from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from num2words import num2words
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 


In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
!python -m spacy download es_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 259.2 kB/s eta 0:00:50
     --------------------------------------- 0.0/12.9 MB 259.2 kB/s eta 0:00:50
     --------------------------------------- 0.1/12.9 MB 326.1 kB/s eta 0:00:40
     --------------------------------------- 0.1/12.9 MB 374.1 kB/s eta 0:00:35
     --------------------------------------- 0.1/12.9 MB 374.1 kB/s eta 0:00:35
     --------------------------------------- 0.1/12.9 MB 374.1 kB/s eta 0:00:35
     --------------------------------------- 0.1/12.9 MB 374.1 kB/s eta 0:00:35
     --------------------------------------- 0.1/12.9 MB 374.1 kB/s eta 0:00:35
     --------------------------------------- 0.1/12.9 MB 218.3 kB/s eta 0:00:59
     --------------------------------------- 0.1/12.9 MB 2

## **2. Perfilamiento y entendimiento de los datos**

### **2.1. Lectura de los datos**

In [6]:
data=pd.read_excel('./data/cat_345.xlsx')
# Asignación a una nueva variable de los datos leidos

### **2.2. Entendimiento de los datos**

In [7]:
data.shape

(3000, 2)

In [ ]:
data.head()

,Textos_espanol,sdg
0,"Por ejemplo, el nÃºmero de consultas externas ...",3
1,"En 2007, el gobierno central financió directam...",3
2,"Claramente, hay muchos otros factores en juego...",3
3,"Por ejemplo, el estado australiano de Victoria...",3
4,"El consumo anual de alcohol se estima en 15,7 ...",3


In [ ]:
data.dtypes

Textos_espanol    object
sdg                int64
dtype: object

In [ ]:
data.describe()

,sdg
count,3000.000000
mean,4.000000
std,0.816633
min,3.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [ ]:
pd.value_counts(data['sdg'])

3    1000
4    1000
5    1000
Name: sdg, dtype: int64

In [ ]:
data.isnull().sum()

Textos_espanol    0
sdg               0
dtype: int64

In [ ]:
data.duplicated(keep = False).sum()

0

## **3. Preparación de los datos**

Se realizaran las siguientes actividades para una adecuada prparación de los datos:



*   Limpieza de los datos.
*   Tokenización.
*   Normalización.

Usando las librerias spacy para el procesamiento de las palabras incluyendo su lematización y verificación de numeros, unicodedata para eliminar caracteres especiales y num2words para convertir numeros en palabras


##**3.1. Limpieza y tokenización**

In [7]:
data_t = data.copy()

In [8]:
nlp = spacy.load('es_core_news_sm')

Se carga el modulo para español de spacy

In [9]:
texts = data_t['Textos_espanol']
tokens = []

stop_words = nlp.Defaults.stop_words  #Stop words en español
cont = 0
for opinion in texts:
  opinionP = opinion.lower() #Se pone el texto en minusculas
  opinionP = unicodedata.normalize('NFKD', opinionP).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  #Se quitan caracteres especiales
  opinionDoc = nlp(opinionP) #Se crea un doc con npl para procesar el texto
  tokensI = []
  for word in opinionDoc:
    wordP = re.sub(r'[^\w\s]', '', word.text) #Remover signos de puntuación
    if wordP != '':
      if wordP == "15.7":
           print(wordP)
      if word.is_digit:
        #print(wordP)

        num_word = num2words(wordP, lang='es')
        #print(num_word)
        tokensI.append(num_word)
        if cont == 5:
           print(wordP)
           print(num_word)

      else:
        if word.text not in stop_words: #No se tienen en cuenta las stop words
            tokensI.append(word.lemma_) #Se toma en cuenta solo el lemma de la palabra
  cont+=1
  tokens.append(tokensI)

2013
dos mil trece


In [28]:
data_t['words'] = pd.Series(tokens, copy=False)
data_t.head()

,Textos_espanol,sdg,words
0,"Por ejemplo, el nÃºmero de consultas externas ...",3,"[ejemplo, naomero, consulta, externo, especial..."
1,"En 2007, el gobierno central financió directam...",3,"[dos mil siete, gobierno, central, financio, d..."
2,"Claramente, hay muchos otros factores en juego...",3,"[claramente, factor, juego, particular, varied..."
3,"Por ejemplo, el estado australiano de Victoria...",3,"[ejemplo, australiano, victoria, agencia, prom..."
4,"El consumo anual de alcohol se estima en 15,7 ...",3,"[consumo, anual, alcohol, estimar, 15.7, litro..."


In [29]:
data_t["words"].dtype

dtype('O')

## **3.2. Normalización**

In [30]:
data_t['words'] = data_t['words'].apply(lambda x: ' '.join(map(str, x)))
data_t

,Textos_espanol,sdg,words
0,"Por ejemplo, el nÃºmero de consultas externas ...",3,ejemplo naomero consulta externo especialista ...
1,"En 2007, el gobierno central financió directam...",3,dos mil siete gobierno central financio direct...
2,"Claramente, hay muchos otros factores en juego...",3,claramente factor juego particular variedad ba...
3,"Por ejemplo, el estado australiano de Victoria...",3,ejemplo australiano victoria agencia promocion...
4,"El consumo anual de alcohol se estima en 15,7 ...",3,consumo anual alcohol estimar 15.7 litro ano h...
...,...,...,...
2995,Un caucus efectivo se basa en fuertes vínculos...,5,caucus efectivo basar fuerte vinculo grupo nac...
2996,"Por el contrario, el porcentaje de hogares en ...",5,contrario porcentaje hogar edad trabajar hombr...
2997,El análisis utilizará una gama de medidas que ...,5,analisis utilizar gama medida reflejar barrera...
2998,"La capacitación económica, el apoyo y, a veces...",5,capacitacion economico apoyo incentivo abordar...


In [31]:
data_t["words"].dtype

dtype('O')

In [32]:
X_data, Y_data = data_t['words'], data_t['sdg'].astype(int)

In [33]:
tf_idf = TfidfVectorizer(max_features=3000)
X_data = tf_idf.fit_transform(X_data)

print(X_data.shape)
X_data.toarray()[0]

(3000, 3000)


array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.05635192])

Para normalizar se usa un vectorizador TF-IDF, quedando la variable predictora en Y_data y los textos en X_data

# **4. Creacion de modelos de clasificación**

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=0)

## **4.1. Primer modelo usando Arboles de descición**

In [38]:
# Inicializar el clasificador de árbol de decisión
clf = DecisionTreeClassifier()

# Entrenar el clasificador
clf.fit(X_train, y_train)

# Predecir las categorías en el conjunto de prueba
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy}')

report = classification_report(y_test, y_pred)
print(report)

Precisión: 0.9183333333333333
              precision    recall  f1-score   support

           3       0.94      0.90      0.92       237
           4       0.90      0.90      0.90       187
           5       0.91      0.95      0.93       176

    accuracy                           0.92       600
   macro avg       0.92      0.92      0.92       600
weighted avg       0.92      0.92      0.92       600



In [40]:
# Se genera la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# Se puede visualizar la matriz de confusión
#plot_confusion_matrix(arbol, X_test, Y_test)  
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot(cmap=plt.cm.Blues)

plt.show()